paragraph level node → select n important paragraphs → determine weight of each paragraph(1 < weight < 5)

In [1]:
!pip install cdlib stanza rouge zss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 228 kB 4.9 MB/s 
     |████████████████████████████████| 691 kB 73.8 MB/s 
     |████████████████████████████████| 2.6 MB 60.9 MB/s 
     |████████████████████████████████| 14.3 MB 48.4 MB/s 
     |████████████████████████████████| 240 kB 77.6 MB/s 
     |████████████████████████████████| 3.3 MB 68.6 MB/s 
     |████████████████████████████████| 174 kB 87.7 MB/s 
     |████████████████████████████████| 2.2 MB 71.1 MB/s 
  Created wheel for zss: filename=zss-1.2.0-py3-none-any.whl size=6745 sha256=75db9cadf912607d38b805b6c6370968916c180df8e13b166269b791fb4ba05c
  Stored in directory: /root/.cache/pip/wheels/9f/cb/21/088cbb8211ba7951845e4b1ed6bc9c2328d8d6823e382ad732
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=4add973b6b7bda59b1c21822f0d5df73a4c9bd8352830171f9d0852732ed48c7
  Stored in directory: /root/.cache/pi

In [2]:
import tensorflow_datasets as tfds
import nltk
import pandas as pd
import nltk.data
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk import word_tokenize, ngrams
import re
import json
#from typing import final
import pandas as pd
import numpy as np
from collections import Counter
import networkx as nx
from cdlib import evaluation, algorithms, NodeClustering
from functools import reduce
import stanza
from nltk import word_tokenize, ngrams
from matplotlib import pyplot as plt
from rouge import Rouge
from zss import simple_distance, Node
import logging

Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'graph_tool', 'karateclub', 'infomap', 'wurlitzer'}
Note: to be able to use all overlapping methods, you need to install some additional packages:  {'karateclub', 'ASLPAw'}
Note: to be able to use all bipartite methods, you need to install some additional packages:  {'leidenalg', 'infomap', 'wurlitzer'}


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
def clean(line):
    # line = line.strip().replace("newline_char", "||||||||||||||||||||||||||||")
    # line = line.strip().replace("newline_char", " ")
    line = line.strip()
    line = line.replace("( opens in new window )", "")
    line = line.replace("click to email this to a friend", "")
    line = line.replace("lick to share on whatsapp", "")
    line = line.replace("click to share on facebook", "")
    line = line.replace("share on facebook", "")
    line = line.replace("click to share on twitter", "")
    line = line.replace("click to share on pinterest", "")
    line = line.replace("click to share on tumblr", "")
    line = line.replace("click to share on google+", "")
    line = line.replace("feel free to share these resources in your social "
                        "media networks , websites and other platforms", "")
    line = line.replace("share share tweet link", "")
    line = line.replace("e-mail article print share", "")
    line = line.replace("read or share this story :", "")
    line = line.replace("share the map view in e-mail by clicking the share "
                        "button and copying the link url .     embed the map "
                        "on your website or blog by getting a snippet of html "
                        "code from the share button .     if you wish to "
                        "provide feedback or comments on the map , or if "
                        "you are aware of map layers or other "
                        "datasets that you would like to see included on our maps , "
                        "please submit them for our evaluation using this this form .", "")
    line = line.replace("share this article share tweet post email", "")
    line = line.replace("skip in skip x embed x share close", "")
    line = line.replace("share tweet pin email", "")
    line = line.replace("share on twitter", "")
    line = line.replace("feel free to weigh-in yourself , via"
                        "the comments section . and while you ’ "
                        "re here , why don ’ t you sign up to "
                        "follow us on twitter us on twitter .", "")
    line = line.replace("follow us on facebook , twitter , instagram and youtube", "")
    line = line.replace("follow us on twitter", "")
    line = line.replace("follow us on facebook", "")
    line = line.replace("play facebook twitter google plus embed", "")
    line = line.replace("play facebook twitter embed", "")
    line = line.replace("enlarge icon pinterest icon close icon", "")
    line = line.replace("follow on twitter", "")
    line = line.replace("autoplay autoplay copy this code to your website or blog", "")
    line = line.replace("|||||", "NEWLINE_CHAR")
    return line

In [5]:
class Data:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.data = []
        self.data_uncleaned = []
        self.data_tokenized = []
        self.labels = []
        self.stemmed_docs = []
        self.ngrams = []
        self.char_ngrams = []
        #self.paragraphs = []

    def read_txt_data(self):
        lines = []

        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

        with open(self.data_dir) as f:
            article = clean(f.read())
            paragraphs = list(para for para in filter(str.strip, article.split("NEWLINE_CHAR")) if len(para) > 10)

            sent_idx = 0

            for para_idx, paragraph in enumerate(paragraphs):

                lines.append({'type': 'para', 'doc': str(tokenizer.tokenize(paragraph)[0]), 'para_idx': para_idx, 'sent_idx': None})
                for line in list(paragraph.split(".")):

                  if len(line) > 5:
                    lines.append({'type': 'sent', 'doc': str(tokenizer.tokenize(line)[0]), 'para_idx': para_idx, 'sent_idx': sent_idx})
                    sent_idx += 1

        self.data = lines

    def para_read_txt_data(self):
        lines = []

        tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
        

        with open(self.data_dir) as f:
            article = clean(f.read())
            paragraphs = list(para for para in filter(str.strip, article.split("NEWLINE_CHAR")) if len(para) > 10)

            for paragraph in paragraphs:
                lines.extend(tokenizer.tokenize(paragraph))
                
        self.data = lines

    def set_data(self):
        data = pd.read_csv(self.data_dir)
        self.data = data.TITLE[:20]
        self.labels = data.CATEGORY

    def build_model(self, stem=True, n=2):
        dirty_docs = []
        cleaned_ngrams = []
        data_tokenized = []

        cleaned_tokenized_docs = []
        stop_words = set(stopwords.words('english'))

        data = self.data # list[dict{'type', 'doc', 'para_idx', 'sent_idx'}]
        #print(data)

        for data_dict in data:
            #doc_idx = document
            doc_type = data_dict['type']
            document = data_dict['doc']
            para_idx = data_dict['para_idx']
            sent_idx = data_dict['sent_idx']

            document = re.sub(r"[^a-z\d ]", "", document.lower())
            dirty_docs.append({'type': doc_type, 'doc': document, 'para_idx': para_idx, 'sent_idx': sent_idx})

            tokens = word_tokenize(document)
            cleaned_vec = [w for w in tokens if w.lower() not in stop_words]

            cleaned_tokenized_docs.append({'type': doc_type, 'doc': cleaned_vec, 'para_idx': para_idx, 'sent_idx': sent_idx})
            if len(cleaned_vec) > 0:
              data_tokenized.append({'type': doc_type, 'doc': cleaned_vec, 'para_idx': para_idx, 'sent_idx': sent_idx})
            cleaned_ngrams.append(list(ngrams(cleaned_vec, n)))

        self.data_tokenized = data_tokenized

        self.data_uncleaned = dirty_docs
        self.ngrams = cleaned_ngrams
        #self.char_ngrams = list(ngrams("".join(self.data).split(), n))

        stemmed = []

        if stem:
            ps = PorterStemmer()

            for doc in self.data_tokenized:
                stem_doc = []

                for word in doc['doc']:
                    stem_doc.append(ps.stem(word))

                #stemmed['doc'] = stem_doc
                stemmed.append({'type': doc['type'], 'doc': stem_doc, 'para_idx': doc['para_idx'], 'sent_idx': doc['sent_idx']})

            self.stemmed_docs = stemmed
            #print(stemmed)

    def para_build_model(self, stem=True, n=2):
        dirty_docs = []
        cleaned_ngrams = []

        cleaned_tokenized_docs = []
        stop_words = set(stopwords.words('english'))

        for document in self.data:
            document = re.sub(r"[^a-z\d ]", "", document.lower())
            dirty_docs.append(document)
            tokens = word_tokenize(document)
            cleaned_vec = [w for w in tokens if w.lower() not in stop_words]
            cleaned_tokenized_docs.append(cleaned_vec)
            cleaned_ngrams.append(list(ngrams(cleaned_vec, n)))

        self.data_tokenized = list(
            filter(lambda x: len(x) > 0, cleaned_tokenized_docs))

        self.data_uncleaned = dirty_docs
        self.ngrams = cleaned_ngrams
        self.char_ngrams = list(ngrams("".join(self.data).split(), n))
        stemmed = []

        if stem:
            ps = PorterStemmer()

            for doc in self.data_tokenized:
                stem_doc = []

                for word in doc:
                    stem_doc.append(ps.stem(word))

                stemmed.append(stem_doc)

            self.stemmed_docs = stemmed
    
    def generate_ngrams(self):
        ngrs = []

        for doc in self.data:
            ngrs.append(list(ngrams(doc, 3)))

        return list(ngrs)

In [6]:
class GNN:
    def __init__(self):
        self.data = []
        self.G = None
        self.G_ngrs = None
        self.pos_tokenized = []

    def para_set_data(self, dir):
        d = Data(dir)
        d.para_read_txt_data()
        d.para_build_model()
        self.data = d.stemmed_docs
        self.orig_data = d.data
        self.data_uncleaned = d.data_uncleaned
        self.data_ngrams = d.ngrams
        self.char_ngrams = d.char_ngrams
        self.data_tokenized = d.data_tokenized

    def set_data(self, dir):
        d = Data(dir)
        d.read_txt_data()
        d.build_model()
        self.data = d.stemmed_docs
        self.orig_data = d.data
        self.data_uncleaned = d.data_uncleaned
        self.data_ngrams = d.ngrams
        self.char_ngrams = d.char_ngrams
        self.data_tokenized = d.data_tokenized

    def doc_sim(self, doc1, doc2):
        len_d1 = len(doc1)
        len_d2 = len(doc2)
        sim_words_set = set(doc1).intersection(set(doc2))
        sim_words_len = len(list(sim_words_set))

        len_d1_log = np.log(len_d1) if len_d1 > 0 else 0
        len_d2_log = np.log(len_d2) if len_d2 > 0 else 0

        len_sum = len_d1_log+len_d2_log
        sim = sim_words_len/len_sum if len_sum > 0.0 and sim_words_len > 0 else 0

        return sim

    @staticmethod
    def get_ted_sim(doc1, doc2):
        return 1/(1+simple_distance(doc1, doc2))


    def build_multilayer_graph(self, metric, para_weight, threshold=0.00001, lamd1=1, lamd2=0):
        # sentence_nodes = [data for data in self.data if data['type'] == 'sent']
        # paragraph_nodes = [data for data in self.data if data['type'] == 'para']

        sentence_indexes = [data['sent_idx'] for data in self.data if data['sent_idx'] != None]
        max_sent_idx = max(sentence_indexes) +1

        para_indexes = [data['para_idx'] for data in self.data]
        max_para_idx = max(para_indexes) +1

        # print('sent: ',max(sentence_indexes), len(sentence_nodes))
        # print('para: ',max(para_indexes), len(paragraph_nodes))

        #print(len(sentence_nodes))

        sentence_similarity_matrix = np.zeros((max_sent_idx, max_sent_idx))

        paragraph_similarity_matrix = np.zeros((max_para_idx, max_para_idx))
        hyperlink_matrix = np.zeros((max_sent_idx, max_para_idx))

        # for i in range(len(sentence_nodes)):
        #   for j in range(i+1, len(sentence_nodes)):
        #     if sentence_nodes[i]['para_idx'] == sentence_nodes[j]['para_idx']:
        #       sentence_similarity_matrix[i][j] = self.doc_sim(sentence_nodes[i]['doc'], sentence_nodes[j]['doc']) * para_weight[sentence_nodes[i]['para_idx']]
        #     else:
        #       sentence_similarity_matrix[i][j] = self.doc_sim(sentence_nodes[i]['doc'], sentence_nodes[j]['doc'])
            
        # for i in range(len(paragraph_nodes)):
        #   for j in range(i+1, len(paragraph_nodes)):
        #     paragraph_similarity_matrix[i][j] = self.doc_sim(paragraph_nodes[i]['doc'], paragraph_nodes[j]['doc'])
        
        # for i in range(len(sentence_nodes)):
        #   for j in range(len(paragraph_nodes)):
        #     if self.doc_sim(sentence_nodes[i]['doc'], paragraph_nodes[j]['doc']) > 0.2:
        #         hyperlink_matrix[i][j] = 1

        #temp = []
        for i in range(len(self.data)):
          for j in range(i+1, len(self.data)): # i+1
            i_type = self.data[i]['type']
            j_type = self.data[j]['type']
            if i_type == 'sent' and j_type == 'sent':
              if self.data[i]['para_idx'] == self.data[j]['para_idx']:
                similarity = self.doc_sim(self.data[i]['doc'], self.data[j]['doc']) * para_weight[self.data[i]['para_idx']]
                if similarity == 0:
                  sentence_similarity_matrix[self.data[i]['sent_idx'], self.data[j]['sent_idx']] = 0.0001
                else:
                  sentence_similarity_matrix[self.data[i]['sent_idx'], self.data[j]['sent_idx']] = similarity
              else:
                similarity = self.doc_sim(self.data[i]['doc'], self.data[j]['doc'])
                if similarity == 0:
                  sentence_similarity_matrix[self.data[i]['sent_idx'], self.data[j]['sent_idx']] = 0.0001
                else:
                  sentence_similarity_matrix[self.data[i]['sent_idx'], self.data[j]['sent_idx']] = self.doc_sim(self.data[i]['doc'], self.data[j]['doc'])

            elif i_type == 'para' and j_type == 'para':
              similarity = self.doc_sim(self.data[i]['doc'], self.data[j]['doc']) #* (para_weight[self.data[i]['para_idx']] + para_weight[self.data[j]['para_idx']])/2
              if similarity == 0:
                paragraph_similarity_matrix[self.data[i]['para_idx']][self.data[j]['para_idx']] = 0.0001
              else:
                paragraph_similarity_matrix[self.data[i]['para_idx']][self.data[j]['para_idx']] = similarity
            
            elif i_type == 'sent' and j_type == 'para':
              if self.doc_sim(self.data[i]['doc'], self.data[j]['doc']) > 0:
                hyperlink_matrix[self.data[i]['sent_idx']][self.data[j]['para_idx']] = 1

            elif i_type == 'para' and j_type == 'sent':
              if self.doc_sim(self.data[i]['doc'], self.data[j]['doc']) > 0:
                hyperlink_matrix[self.data[j]['sent_idx']][self.data[i]['para_idx']] = 1

        sent_adj_matrix = sentence_similarity_matrix / np.sum(sentence_similarity_matrix)
        para_adj_matrix = (hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T) / np.sum(hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T)
        final_adj_matrix = sent_adj_matrix*lamd1 + para_adj_matrix*lamd2

        #print(sentence_similarity_matrix.shape, hyperlink_matrix.shape, paragraph_similarity_matrix.shape)
        #final_adj_matrix = sentence_similarity_matrix * lamd1 + hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T * lamd2
        #print(final_adj_matrix.shape)
        #print(np.max(sentence_similarity_matrix* lamd1), np.min(sentence_similarity_matrix* lamd1), np.mean(np.mean(sentence_similarity_matrix* lamd1)))
        #print(np.max(hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T* lamd2), np.min(hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T* lamd2), np.mean(hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T* lamd2))
        #print(np.mean(sentence_similarity_matrix / np.sum(sentence_similarity_matrix)))
        #print(np.mean((hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T) / np.sum(hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T)))
        #print(np.mean(sentence_similarity_matrix* lamd1) , np.mean(hyperlink_matrix @ paragraph_similarity_matrix @ hyperlink_matrix.T* lamd2))

        G = nx.Graph()
        G.add_nodes_from(list(range(max_sent_idx)))

        
        weights = []
        for i in range(final_adj_matrix.shape[0]):
          for j in range(final_adj_matrix.shape[1]):
            if final_adj_matrix[i, j] != 0:
              weights.append(((i, j), final_adj_matrix[i][j]))
    

        weights = list(sorted(
            weights, key=lambda x: x[1], reverse=True if metric == "overlap" else False))
        cutoff = int(len(weights) * threshold)


        #print(weights[:cutoff])
        #print('weights: ', len(weights), print(len(weights[:cutoff])))
        #print('temp: ', len(temp))

        for edge, weight in weights[:cutoff]:
            G.add_edge(*edge, weight=weight)

        self.G = G

  

    def build_graph(self, metric, threshold=0.00001):
        # G.add_edges_from([(1,2,{'color':'blue'}), (2,3,{'weight':8})])
        # G.add_edge(1, 2, weight=4.7 )

        nodes = list(range(len(self.data)))
        
        G = nx.Graph()

        G.add_nodes_from(nodes)

        if metric == "ted":
            deps = self.build_dependency_trees()

        weights = []

        for i in range(0, len(self.data)):
            for j in range(i+1, len(self.data)):

                if metric == "overlap":
                    weights.append(((i, j), self.doc_sim(
                        self.data[i], self.data[j])))
                else:
                    weights.append(
                        ((i, j), self.get_ted_sim(deps[i], deps[j])))

        weights = list(sorted(
            weights, key=lambda x: x[1], reverse=True if metric == "overlap" else False))
        cutoff = int(len(weights) * threshold)

        for edge, weight in weights[:cutoff]:
            G.add_edge(*edge, weight=weight)

        self.G = G

    @staticmethod
    def log_graphs(graph):
        print(
            f"Graph has {graph.number_of_nodes()} nodes, {graph.number_of_nodes()} edges.")

    def build_graph_ngrams(self):

        G = nx.MultiGraph()
        G.add_nodes_from(list(self.char_ngrams[0]))

        for item in self.char_ngrams[1:]:
            G.add_node(item[1])

        G.add_edges_from(self.char_ngrams)

        print(G.nodes)
        self.G_ngrs = G
        self.log_graphs(self.G_ngrs)

    def build_word_graph(self, data):
        g = nx.DiGraph()
        g.add_nodes_from(["S", "E"])

        last_tokens = []
        edges_to_be_added = []

        for idx, sentence in enumerate(data):
            nodes = list(g.nodes)
            tokens = word_tokenize(sentence)

            queue = []

            for token in tokens:
                if token not in queue:
                    queue.append(token)

            g.add_nodes_from(queue)

            edges_to_be_added.append(("S", tokens[0]))
            edges_to_be_added.append((tokens[-1], 'E'))

            ngrams_list = list(ngrams(tokens, 2))
            edges_to_be_added.extend(ngrams_list)

            last_tokens.append(tokens[0])

            edges_to_be_added.append((last_tokens[idx-1], tokens[0]))

        unique, cnts = np.unique(edges_to_be_added, return_counts=True, axis=0)

        for idx, edge in enumerate(unique):
            g.add_edge(*edge, weight=(1/cnts[idx]))

        return g

    @staticmethod
    def sort_clique_items_by_closeness(closeness, cliques):
        cands = []

        for clique in list(filter(lambda x: len(x) > 1, cliques)):
            mapped = map(lambda x: (x, closeness[x]), clique)
            top = list(sorted(mapped, key=lambda x: x[1]))

            idx = 0
            top_cand = top[idx]

            while True:
                if top_cand not in cands:
                    cands.append(top_cand)
                    break
                else:

                    idx += 1
                    if idx < len(top):
                        top_cand = top[idx]
                    else:
                        break

        return {k: v for k, v in cands}

    def summarize(self, sorting_method="pagerank"):
        if sorting_method == "pagerank":
            try:
                scores = nx.pagerank(self.G, max_iter=1000)
            except:
                scores = {}
        elif sorting_method == "hits":
            try:
                scores = nx.hits(self.G, max_iter=1000)[0]
            except:
                scores = {}
        elif sorting_method == "closeness":
            scores = nx.closeness_centrality(self.G)
        elif sorting_method == "betweenness":
            scores = nx.betweenness_centrality(self.G)
        elif sorting_method == "degree":
            scores = nx.degree_centrality(self.G)
        elif sorting_method == "cliques":
            closeness = nx.closeness_centrality(self.G)
            cliques = list(nx.find_cliques(self.G))
            scores = self.sort_clique_items_by_closeness(closeness, cliques)

        sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        final_doc = []

        #print(self.orig_data)
        # i = 0
        # for idx, _ in sorted_docs:
        #     if self.orig_data[idx]['type'] == 'sent':
        #       i+=1
        #       final_doc.append(self.orig_data[idx]['doc'])

        #       if i == 5:
        #         break
        #print(self.orig_data[:5])

        #self.orig_data = [data in data for self.orig_data if data['type'] == 'sent']
        orig_data = [data for data in self.orig_data if data['type'] == 'sent']
        num = 5
        i = 0
        for idx, _ in sorted_docs:
            if i == num:
              break
            try:
              final_doc.append(orig_data[idx]['doc'])
              i += 1
            except:
              i -= 1
              continue

        return ".".join(final_doc)
    
    def paragraph_weight(self, sorting_method="pagerank"):
        if sorting_method == "pagerank":
            try:
                scores = nx.pagerank(self.G, max_iter=1000)
            except:
                scores = {}
        elif sorting_method == "hits":
            try:
                scores = nx.hits(self.G, max_iter=1000)[0]
            except:
                scores = {}
        elif sorting_method == "closeness":
            scores = nx.closeness_centrality(self.G)
        elif sorting_method == "betweenness":
            scores = nx.betweenness_centrality(self.G)
        elif sorting_method == "degree":
            scores = nx.degree_centrality(self.G)
        elif sorting_method == "cliques":
            closeness = nx.closeness_centrality(self.G)
            cliques = list(nx.find_cliques(self.G))
            scores = self.sort_clique_items_by_closeness(closeness, cliques)

        sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)

        para_weight = {}

        for i, (para_idx, _) in enumerate(sorted_docs):
            if i <= len(sorted_docs) / 5:
              para_weight[para_idx] = 5
            elif i <= len(sorted_docs) / 5 * 2:
              para_weight[para_idx] = 4
            elif i <= len(sorted_docs) / 5 * 3:
              para_weight[para_idx] = 3
            elif i <= len(sorted_docs) / 5 * 4:
              para_weight[para_idx] = 2
            else:
              para_weight[para_idx] = 1

        return para_weight

    def read_docs_by_idx(self, collection):
        final_doc = []

        for idx in collection:

            final_doc.append(self.orig_data[idx])

        return ". ".join(final_doc)

    @staticmethod
    def tree_to_graph(deps):
        g = nx.Graph()

        heads = []
        ids = []

        for dep in deps:
            heads.append(dep['head'])
            ids.append(dep['id'])

        g.add_nodes_from(ids)

        for idx, _ in enumerate(ids):
            if heads[idx] != 0:
                g.add_edge(ids[idx], heads[idx])

        return g

    @staticmethod
    def build_zss_tree(deps):
        labels = {}
        tree = {}
        root = None

        for dep in deps:
            if dep['head'] == 0:
                root = dep['id']
                labels[dep['id']] = dep['text']

            if True:
                if dep['head'] in tree:
                    tree[dep['head']].append(dep['id'])
                else:
                    tree[dep['head']] = [dep['id']]

                labels[dep['id']] = dep['text']

        def iterative(coll):
            out = []
            for child in coll:
                if child in tree:
                    out.append(Node(labels[child], iterative(tree[child])))
                else:
                    out.append(Node(labels[child], []))

            return out

        if len(deps) > 1:
            final_tree = Node(labels[root], iterative(tree[root]))
        else:
            final_tree = Node(labels[1], [])

        return final_tree

    def build_dependency_trees(self):
        deps = []
        term_freqs = []
        nlp = stanza.Pipeline(
            processors="tokenize, pos, lemma, depparse", tokenize_pretokenized=True, verbose=False)

        doc = nlp(self.data_tokenized)

        for idx, sent in enumerate(doc.to_dict()):
            deps.append(self.build_zss_tree(sent))

        return deps

    def evaluate(self, methods, true_summary):
        results = []

        for method in methods:
            summary = gnn.summarize(sorting_method=method)
            #print(summary)
            summary = summary if summary != "" else " "
            results.append(Rouge().get_scores(summary, true_summary))

        return results

In [7]:


methods = ["pagerank", "closeness"]
for i in range(2):
    print(i)
    gnn = GNN()
    gnn.para_set_data(f'./drive/MyDrive/multi_news/{i}_article.txt')
    gnn.build_graph(metric="overlap", threshold=0.1)

    f = open(f"./drive/MyDrive/multi_news/{i}_summ.txt").readlines()

    ## select n most important paragraphs
    para_weight = gnn.paragraph_weight(sorting_method='pagerank')
    print(para_weight)

    gnn = GNN()
    gnn.set_data(f'./drive/MyDrive/multi_news/{i}_article.txt')

    #raise NotImplementedError

    lamd1 = 1
    lamd2 = 0
    gnn.build_multilayer_graph(metric="overlap",
                               threshold=0.1,
                               para_weight = para_weight,
                               lamd1 = lamd1,
                               lamd2 = lamd2)

    result = gnn.evaluate(methods, f[0])
    print(result)

0
{7: 5, 52: 5, 40: 5, 83: 5, 11: 5, 56: 5, 8: 5, 53: 5, 6: 5, 51: 5, 79: 5, 34: 5, 4: 5, 49: 5, 47: 5, 2: 5, 76: 5, 31: 5, 81: 4, 21: 4, 66: 4, 3: 4, 18: 4, 63: 4, 44: 4, 87: 4, 48: 4, 13: 4, 58: 4, 72: 4, 27: 4, 16: 4, 61: 4, 5: 4, 17: 4, 24: 4, 28: 3, 29: 3, 35: 3, 42: 3, 50: 3, 62: 3, 69: 3, 73: 3, 74: 3, 80: 3, 85: 3, 19: 3, 64: 3, 70: 3, 25: 3, 10: 3, 55: 3, 22: 2, 67: 2, 57: 2, 12: 2, 23: 2, 68: 2, 30: 2, 75: 2, 38: 2, 14: 2, 59: 2, 37: 2, 41: 2, 84: 2, 43: 2, 86: 2, 0: 2, 45: 2, 15: 1, 60: 1, 32: 1, 77: 1, 33: 1, 78: 1, 9: 1, 54: 1, 20: 1, 65: 1, 82: 1, 71: 1, 26: 1, 36: 1, 39: 1, 1: 1, 46: 1}
[[{'rouge-1': {'r': 0.2767857142857143, 'p': 0.3522727272727273, 'f': 0.30999999507200005}, 'rouge-2': {'r': 0.061224489795918366, 'p': 0.07563025210084033, 'f': 0.06766916798773284}, 'rouge-l': {'r': 0.25, 'p': 0.3181818181818182, 'f': 0.2799999950720001}}], [{'rouge-1': {'r': 0.23214285714285715, 'p': 0.36619718309859156, 'f': 0.28415300071545885}, 'rouge-2': {'r': 0.047619047619047616,

In [9]:
from tqdm import tqdm
for l in [0.2]:
  lamd1 = l
  lamd2 = 1 - l
  batch_size = 30

  print("================================")
  print('lamds: ', lamd1, lamd2)

  methods = ["pagerank"] #, "closeness", "hits", "betweenness", "degree", "cliques"
  results = [[] for _ in range(len(methods))]

  logging.basicConfig(filename='results2.log',
                      level=logging.DEBUG, format='%(asctime)s %(message)s')

  batches = 100

  avg_results_f = [[[], [], []] for _ in range(len(methods))]
  avg_results_r = [[[], [], []] for _ in range(len(methods))]

  # for testing all at once and evaluating all at once

  for i in tqdm(range(3001)):
      #print(i)
      # measure paragraph importance weight
      para_gnn = GNN()
      para_gnn.para_set_data(f'./drive/MyDrive/multi_news/{i}_article.txt')
      para_gnn.build_graph(metric="overlap", threshold=0.1)

      f = open(f"./drive/MyDrive/multi_news/{i}_summ.txt").readlines()

      para_weight = para_gnn.paragraph_weight(sorting_method='pagerank') #para idx:para_importance_weight

      #print(para_weight)
      
      gnn = GNN()
      gnn.set_data(f'./drive/MyDrive/multi_news/{i}_article.txt')

      #raise NotImplementedError

      gnn.build_multilayer_graph(metric="overlap",
                                threshold=0.1,
                                para_weight = para_weight,
                                lamd1 = lamd1,
                                lamd2 = lamd2)

      result = gnn.evaluate(methods, f[0])
      #print(result)

      logging.info(result)

      for idx in range(len(methods)):
          results[idx].append(result[idx])

  for idx in range(len(methods)):
      r1, r2, rl = [], [], []
      f1, f2, fl = [], [], []

      for item in results[idx]:
          r1_score = item[0]['rouge-1']['r']
          r2_score = item[0]['rouge-2']['r']
          rl_score = item[0]['rouge-l']['r']

          if r1 != 0.0:
              r1.append(r1_score)

          if r2 != 0.0:
              r2.append(r2_score)

          if rl != 0.0:
              rl.append(rl_score)

          f1_score = item[0]['rouge-1']['f']
          f2_score = item[0]['rouge-2']['f']
          fl_score = item[0]['rouge-l']['f']

          if f1 != 0.0:
              f1.append(f1_score)

          if f2 != 0.0:
              f2.append(f2_score)

          if fl != 0.0:
              fl.append(fl_score)

      r1 = sum(r1)/len(r1)
      r2 = sum(r2)/len(r2)
      rl = sum(rl)/len(rl)

      f1 = sum(f1)/len(f1)
      f2 = sum(f2)/len(f2)
      fl = sum(fl)/len(fl)

      print(methods[idx], f"{r1} / {r2} / {rl} \n", f"{f1} / {f2} / {fl} \n\n")


  # for testing in different batches

  for size in tqdm(range(1, batches + 1)):

      for i in range((size-1)*batch_size, ((size-1)*batch_size)+batch_size):
          # print(i)
          gnn = GNN()
          gnn.set_data(f'./drive/MyDrive/multi_news/{i}_article.txt')
          gnn.build_graph(metric="overlap", threshold=0.1)

          f = open(f"./drive/MyDrive/multi_news/{i}_summ.txt").readlines()
          result = gnn.evaluate(methods, f[0])
          logging.info(result)

          for idx in range(len(methods)):
              results[idx].append(result[idx])

      for idx in range(len(methods)):
          r1, r2, rl = [], [], []
          f1, f2, fl = [], [], []

          for item in results[idx]:
              r1_score = item[0]['rouge-1']['r']
              r2_score = item[0]['rouge-2']['r']
              rl_score = item[0]['rouge-l']['r']

              if r1 != 0.0:
                  r1.append(r1_score)

              if r2 != 0.0:
                  r2.append(r2_score)

              if rl != 0.0:
                  rl.append(rl_score)

              f1_score = item[0]['rouge-1']['f']
              f2_score = item[0]['rouge-2']['f']
              fl_score = item[0]['rouge-l']['f']

              if f1 != 0.0:
                  f1.append(f1_score)

              if f2 != 0.0:
                  f2.append(f2_score)

              if fl != 0.0:
                  fl.append(fl_score)

          r1 = sum(r1)/len(r1)
          r2 = sum(r2)/len(r2)
          rl = sum(rl)/len(rl)

          f1 = sum(f1)/len(f1)
          f2 = sum(f2)/len(f2)
          fl = sum(fl)/len(fl)

          avg_results_r[idx][0].append(r1)
          avg_results_r[idx][1].append(r2)
          avg_results_r[idx][2].append(rl)

          avg_results_f[idx][0].append(f1)
          avg_results_f[idx][1].append(f2)
          avg_results_f[idx][2].append(fl)

  for idx in range(len(methods)):
      print("f for", methods[idx], sum(avg_results_f[idx][0])/len(avg_results_f[idx][0]), sum(avg_results_f[idx]
                                                                                              [1])/len(avg_results_f[idx][1]), sum(avg_results_f[idx][1])/len(avg_results_r[idx][1]))
      print("r for", methods[idx], sum(avg_results_r[idx][0])/len(avg_results_r[idx][0]), sum(avg_results_r[idx]
                                                                                              [1])/len(avg_results_r[idx][1]), sum(avg_results_r[idx][2])/len(avg_results_r[idx][2]))


lamds:  0.2 0.8


100%|██████████| 3001/3001 [21:20<00:00,  2.34it/s]


pagerank 0.23529089538374062 / 0.07169775263483197 / 0.2067631024709254 
 0.2804930232049544 / 0.08524969711883346 / 0.24687648257708739 




100%|██████████| 100/100 [11:17<00:00,  6.78s/it]

f for pagerank 0.26862098515365257 0.08104979765925135 0.08104979765925135
r for pagerank 0.22020368084670253 0.06638719362459866 0.1942800363464616
